In [30]:
import pandas as pd
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from copy import deepcopy

In [48]:
class gql_data_processor:
    
    def __init__(self,url,verify=True,retries=5):
        
        #define url endpoint for transport
        transport=RequestsHTTPTransport(url=url,
                                        verify=verify,
                                        retries=retries)
        
        #Create a GraphQL client using the defined transport
        self.client = Client(transport=transport,fetch_schema_from_transport=True)
        
    def query(self,script):
        
        query = gql(script)
        response = self.client.execute(query)
        return response
    
    def query_blocks(self,script,latest_block,block_lookback,step,entity,**kwargs):
        
        responses = []
        
        current_block = deepcopy(latest_block)
        while current_block > (latest_block-block_lookback):
            response = self.query(script)[entity]
            response['block'] = current_block
            responses.append(response)
            current_block = (current_block-step)
            
        self.df = pd.DataFrame(responses)

In [53]:
url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
latest_block = 12504845
block_lookback = 5000
step = 10
entity = 'uniswapFactory'
query = '''
            query {
            uniswapFactory(id: "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f", block: {number: ''' + str(latest_block) + '''}) {
                totalVolumeUSD
                totalLiquidityUSD
                txCount
            }
            }
            '''

In [54]:
uniswap_liquidity = gql_data_processor(url)
uniswap_liquidity.query_blocks(query,latest_block,block_lookback,step,entity)

In [76]:
uniswap_liquidity.df.head()

,totalLiquidityUSD,totalVolumeUSD,txCount,block
0,5068976844.268130309089611211901216,270485698473.5403427235153598532438,48435740,12504845
1,5068976844.268130309089611211901216,270485698473.5403427235153598532438,48435740,12504835
2,5068976844.268130309089611211901216,270485698473.5403427235153598532438,48435740,12504825
3,5068976844.268130309089611211901216,270485698473.5403427235153598532438,48435740,12504815
4,5068976844.268130309089611211901216,270485698473.5403427235153598532438,48435740,12504805
